# **Analiza ekonomskega stanja slovenskih občin**
#### *Projektna naloga iz analize podatkov pri predmetu Programiranje 1*

> V svoji analizi podatkov predstavljam določene aspekte ekonomskega stanja v 210 slovenskih občinah glede na zbirko podatkov [statističnega urada Republike Slovenije](https://www.stat.si/obcine) za leto 2019. V Sloveniji od leta 2011 obstaja 212 občin - zadnji dve novonastali, Anakaran in Mirna, sta iz te analize izvzeti.
>
> #### **Zajem podatkov**
> Za vsako občino sem zajel naslednje podatke:
> * ime občine
> * regija in upravna enota
> * površina, število prebivalcev in skupni prirast prebivalstva
> * stopnja delovno aktivnega prebivalstva
> * neto plača na prebivalca
>
> #### **Delovne hipoteze**
> Pred začetkom sem si postavil nekaj hipotez oziroma vprašanj, na katera bom poskusil kar se dosledno odgovoriti in jih kasneje še obrazložiti:
> * Ali obstaja korelacija med povprečno neto plačo in letnim prirastom?
> * Kje se pojavljajo skrajni primeri stopnje delovno aktivnega prebivalstva?
> * Kakšni so prihodki glede na regijo in upravno enoto?


### **Delovno okolje**

Analiza in manipulacija podatkov bo izvedena z uporabo knjižnice Pandas, zato je potrebno le-to najprej uvoziti. Za upravljanje in reprezentacijo rezultatov pa bomo uporabljali še nekatera priročna orodja.

In [27]:
import pandas as pd
import os.path

Naložimo še razpredelnico podatkov, s katerimi bomo razpolagali.

In [28]:
obcine = pd.read_csv('obcine.csv', index_col='id')

### **Proučevanje podatkov**

### **Interpretacija rezultatov**

### **Zaključek**